In [1]:
from quant_met import hamiltonians, utils
import numpy as np
import pandas as pd
import time
from scipy import optimize

In [5]:
lattice_constant = np.sqrt(3)

all_K_points = 4 * np.pi / (3 * lattice_constant) * np.array([
    (np.sin(i * np.pi / 6), np.cos(i * np.pi / 6)) for i in [1, 3, 5, 7, 9, 11]]
)

BZ_grid = utils.generate_uniform_grid(20, 20, all_K_points[1], all_K_points[5], origin=np.array([0, 0]))

In [6]:
def minimize(mu, V):
    gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
    gap_size_vs_U['U'] = U_range

    for i, U in enumerate(U_range):
        egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U, U_x=U)

        start = time.time()
        solution = optimize.brute(
            func=hamiltonians.free_energy,
            args=(
                egx_h,
                BZ_grid,
            ),
            ranges=[(-2, 2) for _ in range(egx_h.number_of_bands)],
            Ns=20,
            workers=-1,
            finish=optimize.fmin,
            #full_output=True,
        )
        print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
        end = time.time()
        print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
        gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution

    gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_U_uniform/V_{V}_mu_{mu}.csv')


In [7]:
U_range = np.linspace(start=0.01, stop=5, num=10)

minimize(mu=0, V=1)
minimize(mu=-0.5, V=1)
minimize(mu=-1, V=1)

minimize(mu=0, V=2)
minimize(mu=-0.5, V=2)
minimize(mu=-2, V=2)

V = 1, mu = 0, U = 0.01, solution: [ 3.74299211e-05 -3.55366920e-05  3.57434056e-05]
Time taken to solve the gap equation: 44.50 seconds
V = 1, mu = 0, U = 0.5644444444444445, solution: [-0.01056551 -0.09294924 -0.18027943]
Time taken to solve the gap equation: 43.74 seconds
V = 1, mu = 0, U = 1.118888888888889, solution: [0.04815316 0.19578106 0.38544496]
Time taken to solve the gap equation: 43.32 seconds
V = 1, mu = 0, U = 1.6733333333333336, solution: [-0.12635972 -0.32369091 -0.61379375]
Time taken to solve the gap equation: 43.83 seconds
V = 1, mu = 0, U = 2.227777777777778, solution: [-0.27011096 -0.50356734 -0.86170617]
Time taken to solve the gap equation: 42.84 seconds
V = 1, mu = 0, U = 2.7822222222222224, solution: [-0.50731806 -0.75818866 -1.12920517]
Time taken to solve the gap equation: 42.71 seconds
V = 1, mu = 0, U = 3.336666666666667, solution: [-0.8208732  -1.0652034  -1.41736137]
Time taken to solve the gap equation: 42.91 seconds
V = 1, mu = 0, U = 3.89111111111111

In [22]:
U_range = np.linspace(start=0.01, stop=5, num=10)

for V in [0.1, 1, 2]:
    for mu in [0, -0.5, -1, -1.5]:
        gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
        gap_size_vs_U['U'] = U_range

        for i, U in enumerate(U_range):
            egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U, U_x=U)

            start = time.time()
            solution = optimize.brute(
                func=hamiltonians.free_energy_uniform_pairing,
                args=(
                    egx_h,
                    BZ_grid,
                    None,
                ),
                ranges=[(0, 1.5)],
                Ns=20,
                workers=-1,
                finish=optimize.fmin,
                #full_output=True,
            )
            print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
            end = time.time()
            print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
            gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution * np.ones(3)

        gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_U_uniform_pairing_constraint/V_{V}_mu_{mu}.csv')

V = 0.1, mu = 0, U = 0.01, solution: [0.]
Time taken to solve the gap equation: 6.60 seconds
V = 0.1, mu = 0, U = 0.5644444444444445, solution: [0.10553043]
Time taken to solve the gap equation: 17.27 seconds
V = 0.1, mu = 0, U = 1.118888888888889, solution: [0.24979441]
Time taken to solve the gap equation: 12.98 seconds
V = 0.1, mu = 0, U = 1.6733333333333336, solution: [0.44278372]
Time taken to solve the gap equation: 15.40 seconds
V = 0.1, mu = 0, U = 2.227777777777778, solution: [0.69387336]
Time taken to solve the gap equation: 15.61 seconds
V = 0.1, mu = 0, U = 2.7822222222222224, solution: [0.9872777]
Time taken to solve the gap equation: 17.93 seconds
V = 0.1, mu = 0, U = 3.336666666666667, solution: [1.29171464]
Time taken to solve the gap equation: 16.58 seconds
V = 0.1, mu = 0, U = 3.8911111111111114, solution: [1.59521484]
Time taken to solve the gap equation: 16.82 seconds
V = 0.1, mu = 0, U = 4.445555555555556, solution: [1.8965332]
Time taken to solve the gap equation:

In [13]:
U_range = np.linspace(start=0.01, stop=5, num=10)

U_gr = 1

for V in [0.1, 1, 2]:
    for mu in [0, -0.5, -1, -1.5]:
        gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
        gap_size_vs_U['U'] = U_range

        for i, U in enumerate(U_range):
            egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U_gr, U_x=U)

            start = time.time()
            solution = optimize.brute(
                func=hamiltonians.free_energy,
                args=(
                    egx_h,
                    BZ_grid,
                    None,
                ),
                ranges=[(0, 2) for _ in range(egx_h.number_of_bands)],
                Ns=20,
                workers=-1,
                finish=optimize.fmin,
                #full_output=True,
            )
            print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
            end = time.time()
            print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
            gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution

        gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_UX/V_{V}_mu_{mu}.csv')

mu = 0, U = 0.01, solution: [7.39694192e-04 5.12763546e-03 2.21835353e-05]
Time taken to solve the gap equation: 42.96 seconds
mu = 0, U = 0.5644444444444445, solution: [0.00597997 0.00751461 0.27917164]
Time taken to solve the gap equation: 45.81 seconds
mu = 0, U = 1.118888888888889, solution: [0.00659303 0.00711829 0.556684  ]
Time taken to solve the gap equation: 44.76 seconds
mu = 0, U = 1.6733333333333336, solution: [0.00660852 0.00690225 0.83412816]
Time taken to solve the gap equation: 43.37 seconds
mu = 0, U = 2.227777777777778, solution: [0.00661346 0.00674641 1.11145268]
Time taken to solve the gap equation: 47.69 seconds
mu = 0, U = 2.7822222222222224, solution: [0.00653957 0.00657567 1.38876919]
Time taken to solve the gap equation: 43.46 seconds
mu = 0, U = 3.336666666666667, solution: [0.00649126 0.00649987 1.66613379]
Time taken to solve the gap equation: 43.75 seconds
mu = 0, U = 3.8911111111111114, solution: [0.0072137  0.00519786 1.94343995]
Time taken to solve the g

In [28]:
U_range = np.linspace(start=0.01, stop=5, num=10)

#for mu in [0, -0.5, -1, -1.5]:
for mu in [-2, -3, -4]:
    gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1'], index=range(len(U_range)))
    gap_size_vs_U['U'] = U_range

    for i, U in enumerate(U_range):
        graphene_h = hamiltonians.GrapheneHamiltonian(t_nn=1, a=lattice_constant, mu=mu, coulomb_gr=U)

        start = time.time()
        solution = optimize.brute(
            func=hamiltonians.free_energy,
            args=(
                graphene_h,
                BZ_grid,
                None,
            ),
            ranges=[(0, 2.5) for _ in range(graphene_h.number_of_bands)],
            Ns=20,
            workers=-1,
            finish=optimize.fmin,
            #full_output=True,
        )
        print(f'mu = {mu}, U = {U}, solution: {solution}')
        end = time.time()
        print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
        gap_size_vs_U.loc[i, ['delta_0', 'delta_1']] = solution

    gap_size_vs_U.to_csv(f'gap_plots_data/graphene/gap_size_vs_U/V_{V}_mu_{mu}.csv')

mu = -2, U = 0.01, solution: [0. 0.]
Time taken to solve the gap equation: 3.50 seconds
mu = -2, U = 0.5644444444444445, solution: [0.00314687 0.0030965 ]
Time taken to solve the gap equation: 4.46 seconds
mu = -2, U = 1.118888888888889, solution: [0.02471661 0.02470941]
Time taken to solve the gap equation: 4.64 seconds
mu = -2, U = 1.6733333333333336, solution: [0.11680057 0.11685294]
Time taken to solve the gap equation: 4.80 seconds
mu = -2, U = 2.227777777777778, solution: [0.29820186 0.29821865]
Time taken to solve the gap equation: 4.76 seconds
mu = -2, U = 2.7822222222222224, solution: [0.53010896 0.53013742]
Time taken to solve the gap equation: 4.87 seconds
mu = -2, U = 3.336666666666667, solution: [0.79026533 0.79029701]
Time taken to solve the gap equation: 5.01 seconds
mu = -2, U = 3.8911111111111114, solution: [1.0681758  1.06823029]
Time taken to solve the gap equation: 4.88 seconds
mu = -2, U = 4.445555555555556, solution: [1.35823609 1.35816651]
Time taken to solve the

In [14]:
U = 0.01

for V in [0.1, 1, 2]:
    for mu in [0, -0.5, -1, -1.5]:
        gap_size_vs_U = pd.DataFrame(columns=['U', 'delta_0', 'delta_1', 'delta_2'], index=range(len(U_range)))
        N_k_range = np.arange(2, 100, step=10)
        gap_size_vs_U['N_k'] = N_k_range

        for i, N_k in enumerate(N_k_range):
            BZ_grid = utils.generate_uniform_grid(N_k, N_k, all_K_points[1], all_K_points[5], origin=np.array([0, 0]))
            egx_h = hamiltonians.EGXHamiltonian(t_gr=1, t_x=0.01, a=lattice_constant, V=V, mu=mu, U_gr=U, U_x=U)

            start = time.time()
            solution = optimize.brute(
                func=hamiltonians.free_energy_uniform_pairing,
                args=(
                    egx_h,
                    BZ_grid,
                    None,
                ),
                ranges=[(0, 0.5)],
                Ns=40,
                workers=-1,
                finish=optimize.fmin,
                #full_output=True,
            )
            print(f'V = {V}, mu = {mu}, U = {U}, solution: {solution}')
            end = time.time()
            print(f'Time taken to solve the gap equation: {end - start:0.2f} seconds')
            gap_size_vs_U.loc[i, ['delta_0', 'delta_1', 'delta_2']] = solution * np.ones(3)

        gap_size_vs_U.to_csv(f'gap_plots_data/egx/gap_size_vs_U_uniform_pairing_constraint_convergence/V_{V}_mu_{mu}.csv')

V = 0.1, mu = 0, U = 0.01, solution: [0.0013125]
Time taken to solve the gap equation: 1.44 seconds
V = 0.1, mu = 0, U = 0.01, solution: [6.25e-05]
Time taken to solve the gap equation: 1.57 seconds
V = 0.1, mu = 0, U = 0.01, solution: [0.]
Time taken to solve the gap equation: 1.97 seconds


Process SpawnPoolWorker-2843:
Process SpawnPoolWorker-2841:
Process SpawnPoolWorker-2848:
Process SpawnPoolWorker-2846:
Process SpawnPoolWorker-2844:
Process SpawnPoolWorker-2842:
Process SpawnPoolWorker-2840:
Process SpawnPoolWorker-2839:
Process SpawnPoolWorker-2847:
Process SpawnPoolWorker-2845:


KeyboardInterrupt: 